In [ ]:
# Milestone 4: Financial Document Segmentation and Parsing
# Dataset: constituents-financials.csv
# Note: Since raw documents are unavailable, we simulate segmentation & table parsing
# using logical grouping of columns from the structured CSV.

import pandas as pd

# Load the dataset
df = pd.read_csv('/content/constituents-financials.csv')

# Clean column names (remove leading/trailing spaces if any)
df.columns = df.columns.str.strip()

# Define logical "document sections" based on financial reporting structure
sections = {
    "Company Information": ["Name", "Sector"],
    "Market Data": ["Price", "52 Week Low", "52 Week High", "Market Cap"],
    "Valuation Metrics": ["Price/Earnings", "Price/Sales", "Price/Book", "EBITDA"],
    "Profitability & Earnings": ["Earnings/Share", "EBITDA"],
    "Dividend Information": ["Dividend Yield"]
}

# Function to simulate "segmentation" — returns a dict of DataFrames per section
def segment_financial_document(dataframe, sections):
    segmented = {}
    for section_name, cols in sections.items():
        # Only include columns that exist in the dataframe
        cols_in_df = [col for col in cols if col in dataframe.columns]
        segmented[section_name] = dataframe[["Name"] + cols_in_df].copy()
    return segmented

# Perform segmentation
segmented_docs = segment_financial_document(df, sections)

# Example: Show a sample of each "section" for one company (e.g., Apple Inc.)
print("=== Simulated Financial Document Segmentation ===\n")

apple_data = df[df['Name'] == "Apple Inc."].iloc[0]

for section_name, cols in sections.items():
    print(f"--- {section_name} ---")
    cols_in_df = [col for col in cols if col in df.columns]
    for col in cols_in_df:
        print(f"{col}: {apple_data[col]}")
    print()

# Simulate "table parsing": extract key financial tables as structured DataFrames

# Table 1: Valuation Summary
valuation_table = df[["Name", "Sector", "Price", "Price/Earnings", "Price/Sales", "Price/Book"]].copy()

# Table 2: Dividend & Income Summary
dividend_income_table = df[["Name", "Dividend Yield", "Earnings/Share"]].copy()

# Table 3: Market Performance
market_table = df[["Name", "52 Week Low", "52 Week High", "Market Cap"]].copy()

# Optional: Save parsed "tables" as separate CSVs (simulating parsed output)
valuation_table.to_csv("parsed_valuation_table.csv", index=False)
dividend_income_table.to_csv("parsed_dividend_income_table.csv", index=False)
market_table.to_csv("parsed_market_performance_table.csv", index=False)

# ------------------------------
# 4. FINAL MODEL EVALUATION (with EDA-style diagnostics)
# ------------------------------
def evaluate_financial_data_quality(df_eval):
    """
    Evaluates the quality of extracted/parsed financial data.
    Metrics: completeness, validity of key ratios, consistency.
    Includes lightweight EDA-style diagnostics for transparency.
    """
    total = len(df_eval)

    # Completeness: % of non-null in critical fields
    critical_fields = ['Price', 'Market Cap', 'Earnings/Share', 'Sector', 'Name']
    completeness = df_eval[critical_fields].notnull().mean().mean()

    # Validity checks:
    # - Price > 0
    # - Market Cap > 0
    valid_price = (df_eval['Price'] > 0).sum()
    valid_mc = (df_eval['Market Cap'] > 0).sum()

    # Consistency: 52-week low <= Price <= 52-week high (allow small tolerance)
    consistent_price_range = (
        (df_eval['52 Week Low'] <= df_eval['Price'] + 1e-5) &
        (df_eval['Price'] <= df_eval['52 Week High'] + 1e-5)
    ).sum()

    # --- EDA-style diagnostics (descriptive stats for key fields) ---
    price_stats = df_eval['Price'].describe().round(2).to_dict()
    pe_ratio_stats = df_eval['Price/Earnings'].describe().round(2).to_dict()
    market_cap_stats = df_eval['Market Cap'].describe().round(2).to_dict()

    # Count of missing values per critical field
    missing_counts = df_eval[critical_fields].isnull().sum().to_dict()

    report = {
        'total_companies': total,
        'data_completeness_ratio': round(completeness, 4),
        'valid_price_entries': valid_price,
        'valid_market_cap_entries': valid_mc,
        'consistent_price_range_entries': consistent_price_range,
        'quality_score': round((completeness +
                                valid_price/total +
                                valid_mc/total +
                                consistent_price_range/total) / 4, 4),
        # EDA Diagnostics
        'eda_price_summary': price_stats,
        'eda_pe_ratio_summary': pe_ratio_stats,
        'eda_market_cap_summary': market_cap_stats,
        'missing_values_per_critical_field': missing_counts
    }
    return report

evaluation_metrics = evaluate_financial_data_quality(df_parsed)
print("\n📊 Final Evaluation Metrics:")
for k, v in evaluation_metrics.items():
    if 'eda_' in k or 'missing_values' in k:
        print(f"   {k}:")
        for subk, subv in v.items():
            print(f"      {subk}: {subv}")
    else:
        print(f"   {k}: {v}")

def extract_company_financials(company_name: str):
    """
    Deployment-ready function to retrieve segmented financials for a given company.
    Simulates an API endpoint.
    """
    row = df_parsed[df_parsed['Name'].str.contains(company_name, case=False, na=False)]
    if row.empty:
        return {"error": f"Company '{company_name}' not found."}
    return segment_financial_report(row.iloc[0])

# Example usage:
print("\n🔍 Example: Extracting financials for 'Apple Inc.'")
result = extract_company_financials("Apple Inc.")
print(result)

print("\n✅ Milestone 4 Completed: Segmentation, Table Parsing, Evaluation & Deployment Ready.")
print("✅ Milestone 4 completed:")
print("- Simulated financial document segmentation into logical sections")
print("- Simulated parsing of financial tables from structured data")
print("- Output tables saved as CSV files (for downstream use)")